# Function call

In [1]:
import openai
from langsmith import wrappers, traceable

# from openai import OpenAI
# client = OpenAI()
client = wrappers.wrap_openai(openai.Client())
# client = wrappers.wrap_openai(OpenAI())

assistant = client.beta.assistants.create(
name="Math Tutor",
instructions="You are a personal math tutor. Write and run code to answer math questions.",
tools=[{"type": "code_interpreter"}],
model="gpt-4o-mini",
)

In [2]:
thread = client.beta.threads.create()

In [3]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [4]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):    
    @override
    def on_text_created(self, text) -> None:
      print(f"\nassistant > ", end="", flush=True)
        
    @override
    def on_text_delta(self, delta, snapshot):
      print(delta.value, end="", flush=True)
        
    def on_tool_call_created(self, tool_call):
      print(f"\nassistant > {tool_call.type}\n", flush=True)
    
    def on_tool_call_delta(self, delta, snapshot):
      if delta.type == 'code_interpreter':
        if delta.code_interpreter.input:
          print(delta.code_interpreter.input, end="", flush=True)
        if delta.code_interpreter.outputs:
          print(f"\n\noutput >", flush=True)
          for output in delta.code_interpreter.outputs:
            if output.type == "logs":
              print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > code_interpreter

import sympy as sp

# Define the variable and the equation
x = sp.symbols('x')
equation = sp.Eq(3*x + 11, 14)

# Solve the equation
solution = sp.solve(equation, x)
solution
The solution to the equation \( 3x + 11 = 14 \) is \( x = 1 \).

# Assistants fundtion calling

https://platform.openai.com/docs/assistants/tools/function-calling

https://docs.smith.langchain.com/

In [5]:
from openai import OpenAI
from langsmith import wrappers, traceable

# client = OpenAI()
client = wrappers.wrap_openai(openai.Client())

assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-4o-mini",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_current_temperature",
                "description": "Get the current temperature for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA",
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["Celsius", "Fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the user's location.",
                        },
                    },
                    "required": ["location", "unit"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_rain_probability",
                "description": "Get the probability of rain for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            },
        },
    ],
)

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather in San Francisco today and the likelihood it'll rain?",
)

from typing_extensions import override
from openai import AssistantEventHandler


class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        # Retrieve events that are denoted with 'requires_action'
        # since these will have our tool_calls
        if event.event == "thread.run.requires_action":
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
        tool_outputs = []

        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "get_current_temperature":
                tool_outputs.append({"tool_call_id": tool.id, "output": "57"})
            elif tool.function.name == "get_rain_probability":
                tool_outputs.append({"tool_call_id": tool.id, "output": "0.06"})

        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
            print()


with client.beta.threads.runs.stream(
    thread_id=thread.id, assistant_id=assistant.id, event_handler=EventHandler()
) as stream:
    stream.until_done()


Today in San Francisco, the temperature is 57°F. The likelihood of rain is only 6%.


In [6]:
from openai import OpenAI
from langsmith import wrappers, traceable

# client = OpenAI()
client = wrappers.wrap_openai(openai.Client())

assistant = client.beta.assistants.create(
    instructions="あなたは店舗運営とデータ分析のスペシャリストです。店舗管理者の相談に答えてください。",
    model="gpt-4o-mini",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "fetch_store_ratings",
                "description": "Fetch store ratings for all stores. Returns: dict: A dictionary where keys are store IDs and values are ratings (1.0 to 5.0).",
                "parameters": {
                    "type": "object",
                    "properties": {},
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "fetch_store_data",
                "description": "Fetch store information for the specified store ID.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "store_id": {
                            "type": "integer",
                            "description": "店舗ID",
                        }
                    },
                    "required": ["store_id"],
                },
            },
        },
    ],
)

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="評価が最も高い店舗を調査し、その店舗名を教えてください。",
)

from typing_extensions import override
from openai import AssistantEventHandler
import json


class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        # Retrieve events that are denoted with 'requires_action'
        # since these will have our tool_calls
        if event.event == "thread.run.requires_action":
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
        tool_outputs = []

        for tool in data.required_action.submit_tool_outputs.tool_calls:
            args = json.loads(tool.function.arguments)
            print(f"function_name: {tool.function.name}")
            print(f"args: {args}")
            if tool.function.name == "fetch_store_ratings":
                output = {
                    "1": 1.3,
                    "2": 4.0,
                    "3": 1.0,
                    "4": 3.5,
                    "5": 3.6,
                    "6": 3.9,
                    "7": 3.5,
                    "8": 4.9,
                    "9": 3.4,
                    "10": 2.0,
                }
                output = json.dumps(output)
                tool_outputs.append({"tool_call_id": tool.id, "output": output})
            elif tool.function.name == "fetch_store_data":
                output = {"id": 8, "name": "モブレストラン", "locale": "東京"}
                output = json.dumps(output)
                tool_outputs.append({"tool_call_id": tool.id, "output": output})

        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
            print()


with client.beta.threads.runs.stream(
    thread_id=thread.id, assistant_id=assistant.id, event_handler=EventHandler()
) as stream:
    stream.until_done()


function_name: fetch_store_ratings
args: {}
function_name: fetch_store_data
args: {'store_id': 8}
評価が最も高い店舗は「ムブレストラン」で、評価は4.9です。店舗の所在地は東京都です。



In [7]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="わたしのさっきの質問を覚えていますか？"
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(messages.data[0].content[0].text.value)
else:
    print(run.status)

はい、あなたの質問は「評価が最も高い店舗を調査し、その店舗名を教えてください。」でした。最も高い評価の店舗は「ムブレストラン」で、評価は4.9です。何か他にお手伝いできることはありますか？


In [8]:
messages.data[0].content[0].text.value

'はい、あなたの質問は「評価が最も高い店舗を調査し、その店舗名を教えてください。」でした。最も高い評価の店舗は「ムブレストラン」で、評価は4.9です。何か他にお手伝いできることはありますか？'